<a href="https://colab.research.google.com/github/drewamorbordelon/DS-Unit-2-Applied-Modeling/blob/master/module3-permutation-boosting/REG_of_monthly_10yrTreasuryModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1087]:
%matplotlib inline
import sys

!pip install category_encoders==2.*

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest

In [1088]:
df = pd.read_csv('/content/MoM_Unit2ProjectDataFrame.csv', parse_dates=['date'], engine='python')
df.head()

,date,cpi,roccpi,roccpi_bps,10y2yr,roc10_2,10y2y_bps,10y,roc10y,10y_bps,vehicle sales,roc_vs,vs_bps,pmi,roc_pmi,rocpmi_bps,crb,dxy,roc_dxy,rocdxy_bps,savings,roc_savings,rocsave_bps,con_conf,vix,currcir,roc_cur,roccur_bps
0,1976-01-01,55.9,NaN,NaN,NaN,NaN,NaN,7.80,NaN,NaN,12.814,NaN,NaN,39.1713,NaN,NaN,NaN,103.88,NaN,NaN,166.6,NaN,NaN,NaN,NaN,84.744,NaN,NaN
1,1976-02-01,56.2,NaN,NaN,NaN,NaN,NaN,7.77,NaN,NaN,13.340,NaN,NaN,39.7897,NaN,NaN,NaN,103.66,NaN,NaN,171.4,NaN,NaN,NaN,NaN,84.038,NaN,NaN
2,1976-03-01,56.5,NaN,NaN,NaN,NaN,NaN,7.66,NaN,NaN,13.378,NaN,NaN,39.8599,NaN,NaN,NaN,105.34,NaN,NaN,175.7,NaN,NaN,NaN,NaN,84.983,NaN,NaN
3,1976-04-01,56.7,NaN,NaN,NaN,NaN,NaN,7.67,NaN,NaN,13.223,NaN,NaN,40.1054,NaN,NaN,NaN,106.23,NaN,NaN,179.0,NaN,NaN,NaN,NaN,86.523,NaN,NaN
4,1976-05-01,57.0,NaN,NaN,NaN,NaN,NaN,7.96,NaN,NaN,12.962,NaN,NaN,40.3213,NaN,NaN,NaN,107.29,NaN,NaN,181.1,NaN,NaN,NaN,NaN,87.327,NaN,NaN


In [1089]:
df['vix_basket'] = df['vix']
df = df.iloc[217:]

In [1090]:
df.head()

,date,cpi,roccpi,roccpi_bps,10y2yr,roc10_2,10y2y_bps,10y,roc10y,10y_bps,vehicle sales,roc_vs,vs_bps,pmi,roc_pmi,rocpmi_bps,crb,dxy,roc_dxy,rocdxy_bps,savings,roc_savings,rocsave_bps,con_conf,vix,currcir,roc_cur,roccur_bps,vix_basket
217,1994-02-01,154.8,2.788845,-14.253446,1.48,-29.857820,-200.393863,6.15,1.990050,1278.817182,15.520,19.669982,538.745273,65.1383,3.196743,-8.187811,118.03,94.55,0.531632,-285.361556,1226.8,3.075113,-22.816457,93.2,14.87,363.041,10.336077,52.713183,14.87
218,1994-03-01,155.3,2.984085,19.524026,1.56,-24.637681,522.013875,6.77,12.271973,1028.192371,15.301,14.820651,-484.933091,66.0163,4.852187,165.544424,118.45,93.06,1.295309,76.367648,1226.4,3.206261,13.114762,91.5,20.45,366.398,10.552979,21.690239,20.45
219,1994-04-01,155.5,2.708058,-27.602676,1.33,-40.090090,-1545.240893,7.06,16.694215,442.224141,16.013,10.229228,-459.142303,66.5980,5.159607,30.742041,122.47,92.12,2.938876,164.356726,1220.8,2.570996,-63.526457,92.6,13.77,370.681,10.571829,1.885001,13.77
220,1994-05-01,155.9,2.700922,-0.713586,1.16,-39.583333,50.675676,7.17,16.396104,-29.811098,14.577,0.399477,-982.975178,67.0655,5.971927,81.232009,130.67,92.52,3.861697,92.282150,1220.6,1.513639,-105.735708,92.8,13.03,373.253,10.471063,-10.076596,13.03
221,1994-06-01,156.4,2.827087,12.616518,1.15,-35.028249,455.508475,7.34,26.551724,1015.562024,15.109,4.271912,387.243512,67.2836,6.548216,57.628823,135.20,89.62,-4.282815,-814.451269,1212.8,0.697443,-81.619668,91.2,14.97,378.375,10.478323,0.726015,14.97


In [1091]:
df['vix_basket'].value_counts()
df.head()

,date,cpi,roccpi,roccpi_bps,10y2yr,roc10_2,10y2y_bps,10y,roc10y,10y_bps,vehicle sales,roc_vs,vs_bps,pmi,roc_pmi,rocpmi_bps,crb,dxy,roc_dxy,rocdxy_bps,savings,roc_savings,rocsave_bps,con_conf,vix,currcir,roc_cur,roccur_bps,vix_basket
217,1994-02-01,154.8,2.788845,-14.253446,1.48,-29.857820,-200.393863,6.15,1.990050,1278.817182,15.520,19.669982,538.745273,65.1383,3.196743,-8.187811,118.03,94.55,0.531632,-285.361556,1226.8,3.075113,-22.816457,93.2,14.87,363.041,10.336077,52.713183,14.87
218,1994-03-01,155.3,2.984085,19.524026,1.56,-24.637681,522.013875,6.77,12.271973,1028.192371,15.301,14.820651,-484.933091,66.0163,4.852187,165.544424,118.45,93.06,1.295309,76.367648,1226.4,3.206261,13.114762,91.5,20.45,366.398,10.552979,21.690239,20.45
219,1994-04-01,155.5,2.708058,-27.602676,1.33,-40.090090,-1545.240893,7.06,16.694215,442.224141,16.013,10.229228,-459.142303,66.5980,5.159607,30.742041,122.47,92.12,2.938876,164.356726,1220.8,2.570996,-63.526457,92.6,13.77,370.681,10.571829,1.885001,13.77
220,1994-05-01,155.9,2.700922,-0.713586,1.16,-39.583333,50.675676,7.17,16.396104,-29.811098,14.577,0.399477,-982.975178,67.0655,5.971927,81.232009,130.67,92.52,3.861697,92.282150,1220.6,1.513639,-105.735708,92.8,13.03,373.253,10.471063,-10.076596,13.03
221,1994-06-01,156.4,2.827087,12.616518,1.15,-35.028249,455.508475,7.34,26.551724,1015.562024,15.109,4.271912,387.243512,67.2836,6.548216,57.628823,135.20,89.62,-4.282815,-814.451269,1212.8,0.697443,-81.619668,91.2,14.97,378.375,10.478323,0.726015,14.97


#Wrangle Function

In [1092]:
def wrangle (df):

  df = df.copy()
  
  # NEW FEATURES = Creating baskets for VIX levels
  # create baskets for VIX
  # >= 30
  # 30-16
  # < 16

  # Creating a new column with categories
  defensive = (df['vix_basket'] >= 30.00)
  tradable = (df['vix_basket'] < 30.00) & (df['vix_basket'] >= 15.00)
  investable = (df['vix_basket'] < 15.00)
  # unknown = (df['vix_basket'] < 30.00) & (df['vix_basket'] > 27.00) & (df['vix_basket'] < 18.00) & (df(['vix_basket'] >= 15.00)

  # NEW Category Features
  df.loc[defensive, 'vix_basket'] = 'defensive'
  df.loc[tradable, 'vix_basket'] = 'tradable'
  df.loc[investable, 'vix_basket'] = 'investable'
  # # df.loc[unknown, 'vix_basket'] = 'unknown'

  # Drop recorded_by (never varies) and id (always varies, random)
  unusable_variance = ['10y2yr', 'roc10_2', '10y2y_bps', 'roc10y', '10y_bps', 
                       'roccpi_bps',	'vs_bps', 'rocsave_bps', 'rocdxy_bps', 
                       'rocpmi_bps', 'roccur_bps',]
   
                      #  'roccpi_bps',	'vs_bps', 'rocsave_bps', 'rocdxy_bps', 
                      #  'rocpmi_bps', 'roccur_bps', 'vix', 'vehicle sales', 
                      #  'roccur_bps', 'roc_vs', 'vs_bps', 'roc_dxy', 
                      #  'roc_savings', 'roc_cur']

  df = df.drop(columns=unusable_variance,  axis=1)  #'cpi', 'gdp', 'rocgdp_bps', 'rocgdp', 'roccpi_bps', 'roccpi','vehicle_sales', 'conjob_claims', 'Indust Prod Index', 'm2vel',, '10y2yr', '10y', 'wti','dxy'
  
  return df

In [1093]:
df = wrangle(df).sort_index()

In [1094]:
df.head()

,date,cpi,roccpi,10y,vehicle sales,roc_vs,pmi,roc_pmi,crb,dxy,roc_dxy,savings,roc_savings,con_conf,vix,currcir,roc_cur,vix_basket
217,1994-02-01,154.8,2.788845,6.15,15.520,19.669982,65.1383,3.196743,118.03,94.55,0.531632,1226.8,3.075113,93.2,14.87,363.041,10.336077,investable
218,1994-03-01,155.3,2.984085,6.77,15.301,14.820651,66.0163,4.852187,118.45,93.06,1.295309,1226.4,3.206261,91.5,20.45,366.398,10.552979,tradable
219,1994-04-01,155.5,2.708058,7.06,16.013,10.229228,66.5980,5.159607,122.47,92.12,2.938876,1220.8,2.570996,92.6,13.77,370.681,10.571829,investable
220,1994-05-01,155.9,2.700922,7.17,14.577,0.399477,67.0655,5.971927,130.67,92.52,3.861697,1220.6,1.513639,92.8,13.03,373.253,10.471063,investable
221,1994-06-01,156.4,2.827087,7.34,15.109,4.271912,67.2836,6.548216,135.20,89.62,-4.282815,1212.8,0.697443,91.2,14.97,378.375,10.478323,investable


In [1095]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 217 to 535
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           318 non-null    datetime64[ns]
 1   cpi            318 non-null    float64       
 2   roccpi         318 non-null    float64       
 3   10y            318 non-null    float64       
 4   vehicle sales  318 non-null    float64       
 5   roc_vs         318 non-null    float64       
 6   pmi            318 non-null    float64       
 7   roc_pmi        318 non-null    float64       
 8   crb            318 non-null    float64       
 9   dxy            318 non-null    float64       
 10  roc_dxy        318 non-null    float64       
 11  savings        318 non-null    float64       
 12  roc_savings    318 non-null    float64       
 13  con_conf       318 non-null    float64       
 14  vix            318 non-null    float64       
 15  currcir        318 no

In [1096]:
# Set `date` as index
df = df.set_index('date')

In [1097]:
df.head()
df.shape

(319, 17)

#Importing GDP quarterly data

In [1098]:
gdp = pd.read_csv('/content/Real Gross Domestic Product (GDPC1) - Copy.csv', parse_dates=['date'], engine='python')
gdp.head(1)

,date,cpi,roccpi,roccpi_bps,gdp,rocgdp,rocgdp_bps
0,1957-01-01,28.7,NaN,NaN,2992.219,NaN,NaN


In [1099]:
# NEW FEATURES = Creating conditional statement for the `Regime`
condition1 = (gdp['roccpi_bps'] < 0.00) & (gdp['rocgdp_bps'] > 0.00)
regime1 = gdp[condition1]
regime1_date = regime1[['date']]
regime1_date['regime'] = 'regime 1'

condition2 = (gdp['roccpi_bps'] > 0.00) & (gdp['rocgdp_bps'] > 0.00)
regime2 = gdp[condition2]
regime2_date = regime2[['date']]
regime2_date['regime'] = 'regime 2'

condition3 = (gdp['roccpi_bps'] > 0.00) & (gdp['rocgdp_bps'] < 0.00)
regime3 = gdp[condition3]
regime3_date = regime3[['date']]
regime3_date['regime'] = 'regime 3'

condition4 = (gdp['roccpi_bps'] < 0.00) & (gdp['rocgdp_bps'] < 0.00)
regime4 = gdp[condition4]
regime4_date = regime4[['date']]
regime4_date['regime'] = 'regime 4'


regime_merge = pd.concat([regime1_date, regime2_date, regime3_date, regime4_date])
gdp = pd.merge(gdp, regime_merge, on='date', how='inner')
gdp.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,date,cpi,roccpi,roccpi_bps,gdp,rocgdp,rocgdp_bps,regime
0,1958-04-01,29.6,2.422145,-36.531112,2925.379,-2.019116,85.316706,regime 1
1,1958-07-01,29.7,2.061856,-36.028966,2993.068,-0.724762,129.435360,regime 1
2,1958-10-01,29.9,2.047782,-1.407410,3063.085,2.659694,338.445615,regime 1
3,1959-01-01,30.0,1.694915,-35.286632,3121.936,7.420567,476.087295,regime 1
4,1959-04-01,30.2,2.027027,33.211177,3192.38,9.127057,170.649002,regime 2


In [1100]:
gdp = gdp.set_index('date')
gdp.head()

,cpi,roccpi,roccpi_bps,gdp,rocgdp,rocgdp_bps,regime
date,,,,,,,
1958-04-01,29.6,2.422145,-36.531112,2925.379,-2.019116,85.316706,regime 1
1958-07-01,29.7,2.061856,-36.028966,2993.068,-0.724762,129.435360,regime 1
1958-10-01,29.9,2.047782,-1.407410,3063.085,2.659694,338.445615,regime 1
1959-01-01,30.0,1.694915,-35.286632,3121.936,7.420567,476.087295,regime 1
1959-04-01,30.2,2.027027,33.211177,3192.38,9.127057,170.649002,regime 2


In [1101]:
gdp = gdp.drop(['cpi', 'roccpi', 'roccpi_bps', 'gdp', 'rocgdp', 'rocgdp_bps'], axis=1)
gdp.head()

,regime
date,
1958-04-01,regime 1
1958-07-01,regime 1
1958-10-01,regime 1
1959-01-01,regime 1
1959-04-01,regime 2


In [1102]:
gdp.shape

(249, 1)

In [1103]:
df = df.join(gdp, how='left')

In [1104]:
df.drop(df.tail(1).index,inplace=True)

In [1105]:
df = df.fillna('month')

In [1106]:
df.tail()

,cpi,roccpi,10y,vehicle sales,roc_vs,pmi,roc_pmi,crb,dxy,roc_dxy,savings,roc_savings,con_conf,vix,currcir,roc_cur,vix_basket,regime
date,,,,,,,,,,,,,,,,,,
2020-03-01,266.795,2.096304,0.70,11.809,-33.768929,100.8026,-5.172839,121.79,99.05,1.819490,10462.7,12.094752,89.1,53.54,1826.076,6.212782,defensive,month
2020-04-01,265.601,1.438310,0.64,9.089,-47.018362,84.5913,-19.722568,117.20,99.57,2.144030,10908.7,17.299111,71.8,34.15,1887.380,9.324036,defensive,regime 4
2020-05-01,265.441,1.236465,0.65,12.440,-30.354943,87.8986,-16.685845,132.24,98.34,0.603581,11269.8,19.939976,72.3,27.51,1929.729,11.401623,tradable,month
2020-06-01,266.065,1.194257,0.66,13.359,-24.695603,94.6167,-10.821074,137.97,97.39,1.310725,11386.8,20.346238,78.1,30.43,1955.370,12.522378,defensive,month
2020-07-01,267.715,1.566086,0.55,15.006,-14.383523,98.3754,-6.918340,143.69,93.35,-5.247665,11425.1,20.247756,72.5,24.46,1978.279,13.245250,tradable,month


In [1107]:
df.corr()

,cpi,roccpi,10y,vehicle sales,roc_vs,pmi,roc_pmi,crb,dxy,roc_dxy,savings,roc_savings,con_conf,vix,currcir,roc_cur
cpi,1.000000,-0.483985,-0.923219,0.019913,-0.092439,0.773437,-0.413175,0.276324,-0.177180,0.085350,0.979431,-0.027865,-0.298372,-0.089176,0.976000,-0.069930
roccpi,-0.483985,1.000000,0.564652,0.273233,-0.090422,-0.394466,0.178362,-0.328202,0.260719,0.034279,-0.439750,-0.399101,0.348775,-0.117958,-0.407896,-0.058824
10y,-0.923219,0.564652,1.000000,0.073703,0.060667,-0.708648,0.457867,-0.305694,0.208826,-0.073483,-0.896879,-0.164625,0.423444,-0.042647,-0.875094,0.018307
vehicle sales,0.019913,0.273233,0.073703,1.000000,0.357153,0.294146,0.338342,-0.232980,0.500231,-0.015700,0.067972,-0.119392,0.671333,-0.439494,0.102058,-0.079017
roc_vs,-0.092439,-0.090422,0.060667,0.357153,1.000000,-0.012795,0.619712,0.034352,-0.054684,-0.062367,-0.050788,0.037523,0.234059,-0.374543,-0.085091,0.108693
pmi,0.773437,-0.394466,-0.708648,0.294146,-0.012795,1.000000,-0.196138,0.544403,-0.095814,-0.009162,0.674188,0.051845,-0.166088,-0.091740,0.682693,-0.333948
roc_pmi,-0.413175,0.178362,0.457867,0.338342,0.619712,-0.196138,1.000000,-0.053028,-0.012913,-0.140944,-0.352788,-0.221406,0.489077,-0.374965,-0.364967,-0.101422
crb,0.276324,-0.328202,-0.305694,-0.232980,0.034352,0.544403,-0.053028,1.000000,-0.684983,-0.206539,0.144891,-0.016943,-0.612633,-0.110845,0.099391,-0.437088
dxy,-0.177180,0.260719,0.208826,0.500231,-0.054684,-0.095814,-0.012913,-0.684983,1.000000,0.349411,-0.136808,0.298859,0.618640,0.166710,-0.075243,0.164016
roc_dxy,0.085350,0.034279,-0.073483,-0.015700,-0.062367,-0.009162,-0.140944,-0.206539,0.349411,1.000000,0.096364,-0.017149,0.155074,0.180798,0.105157,0.121493


#Split TV from FM

In [1108]:
# target is the regime column
# categorical
target = '10y'
# target ='crb'
y = df[target]
X = df.drop(target , axis=1)

In [1109]:
cutoff1 = '2016-01-01'
cutoff2 = '2018-01-01'
# cutoff3 = '2020-01-01'
mask1 = df.index < cutoff1
# mask1 = (df.index <= cutoff1)
mask2 = (df.index >= cutoff1) & (df.index < cutoff2)
mask3 = (df.index >= cutoff2) 
train = df[mask1]
# val = df[~mask1]
val = df[mask2]
test = df[mask3]

In [1110]:
y_train = train[target]
X_train = train.drop([target], axis=1)

y_val = val[target]
X_val = val.drop(target, axis=1)

y_test = test[target]
X_test = test.drop(target, axis=1)

###Test with assert

In [1111]:
# assert df.shape[0] == train.shape[0] + val.shape[0]
assert df.shape[0] == train.shape[0] + val.shape[0] + test.shape[0]

#Establish Baseline

In [1112]:
# Regression --> mean absolute error
from sklearn.metrics import mean_absolute_error
 
print('Baseline MAE:', mean_absolute_error(y_train, [y_train.mean()]*len(y_train)))
# print('Baseline Accuracy:', y_train.value_counts(normalize=True).max())


Baseline MAE: 1.3125223727392332


#Build Model

In [1113]:
model = make_pipeline(
    OneHotEncoder(use_cat_names=True),   #OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    SelectKBest(k=12),
    LinearRegression()  #LinearRegression()  RandomForestRegressor(random_state=42)
)

model.fit(X_train, y_train);

In [1114]:
print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model.predict(X_test)))

Training MAE: 0.376084770336333
Validation MAE: 0.2697139208603627
Test MAE: 0.5926191421277192


In [1115]:
# y_pred = model.predict(X_val)
# print('Accuracy Score:', accuracy_score(y_val, y_pred))

In [1116]:
# print('Training MAE:', mean_absolute_error(y_train, model.predict(X_train)))
# print('Validation MAE:', mean_absolute_error(y_val, model.predict(X_val)))
# print('Test MAE:', mean_absolute_error(y_test, model.predict(X_test)))

In [1117]:
from sklearn.metrics import r2_score

print("Training R2:", model.score(X_train, y_train))
print("Validation R2:", model.score(X_val, y_val))
print("Test R2:", model.score(X_test, y_test))

Training R2: 0.9092971561526869
Validation R2: -0.1316116540051906
Test R2: 0.22574717435381617


In [1118]:
model_rfr = make_pipeline(
    OneHotEncoder(use_cat_names=True),   # OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    RandomForestRegressor(random_state=42,)
)                                  #SelectKBest(k=18), # <-- Hyperparameter # <-- max_depth, n_estimators
model_rfr.fit(X_train, y_train);#XGBRegressor(n_jobs=-1, n_estimators=15)

In [1119]:
print('Training MAE:', mean_absolute_error(y_train, model_rfr.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model_rfr.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model_rfr.predict(X_test)))

Training MAE: 0.07958022813688197
Validation MAE: 0.3076541666666668
Test MAE: 0.7131354838709677


In [1120]:
print("Training R2:", model_rfr.score(X_train, y_train))
print("Validation R2:", model_rfr.score(X_val, y_val))
print("Test R2:", model_rfr.score(X_test, y_test))

Training R2: 0.9958166276014386
Validation R2: -0.43981255642488515
Test R2: -0.003914479472967525


In [1121]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor, XGBClassifier

model_gb = make_pipeline(
    OneHotEncoder(use_cat_names=True),   # OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    GradientBoostingRegressor()
)                                  #SelectKBest(k=18), # <-- Hyperparameter # <-- max_depth, n_estimators
model_rfr.fit(X_train, y_train);


In [1122]:
print('Training MAE:', mean_absolute_error(y_train, model_gb.predict(X_train)))
print('Validation MAE:', mean_absolute_error(y_val, model_gb.predict(X_val)))
print('Test MAE:', mean_absolute_error(y_test, model_gb.predict(X_test)))

ValueError: ignored

In [ ]:


rfr_model = make_pipeline(
    OneHotEncoder(use_cat_names=True),     #OneHotEncoder(use_cat_names=True),
    SimpleImputer(strategy='mean'),
    SelectKBest(k=7),
    # XGBRegressor(n_jobs=-1, n_estimators=100),
    RandomForestRegressor(random_state=42)
)
# The ranges I want to test as dictionary
params= {'randomforestregressor__n_estimators': range(50, 201, 50),
         'randomforestregressor__max_depth': range(5, 26, 10)}

# Create Grid Search
gs = GridSearchCV(rfr_model,
                  param_grid = params,
                  n_jobs=-1,
                  verbose=1, 
                  cv=10
)

In [ ]:
gs.fit(X_train, y_train);

In [ ]:
# # What are the best set of hyperparameters?
gs.best_params_

In [ ]:
# What if I want to save my best model?
best_model = gs.best_estimator_

In [ ]:
print('Tuned training MAE:', mean_absolute_error(y_train, gs.predict(X_train)))
print('Tuned validation MAE:', mean_absolute_error(y_val, gs.predict(X_val)))
print('Tuned test MAE:', mean_absolute_error(y_test, gs.predict(X_test)))

In [ ]:
# # Multiple Linear Regression
from sklearn.metrics import r2_score


print("Training R2:", best_model.score(X_train, y_train))
print("Validation R2:", best_model.score(X_val, y_val))
print("Test R2:", best_model.score(X_test, y_test))

In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
X, y = make_classification(n_samples=1000,
                           n_features=10,
                           n_informative=3,
                           n_redundant=0,
                           n_repeated=0,
                           n_classes=2,
                           random_state=0,
                           shuffle=False)

In [ ]:
forest = ExtraTreesClassifier(n_estimators=250,
                              random_state=0)

In [ ]:
forest.fit(X, y)
importances = forest.feature_importances_
std = np.std([tree.feature_importances_ for tree in forest.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

In [ ]:
# Print the feature ranking
# print("Feature ranking:")

# for f in range(X.shape[1]):
#     print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# # Plot the impurity-based feature importances of the forest
# plt.figure()
# plt.title("Feature importances")
# plt.bar(range(X.shape[1]), importances[indices],
#         color="r", yerr=std[indices], align="center")
# plt.xticks(range(X.shape[1]), indices)
# plt.xlim([-1, X.shape[1]])
# plt.show()

In [ ]:
# plot feature importance using built-in function
# from numpy import loadtxt
# from xgboost import XGBRegressor
# from xgboost import plot_importance

# # fit model no training data
# model = XGBRegressor()
# model.fit(X, y)
# # plot feature importance
# plot_importance(model)
# plt.show()